In [1]:
from uAI_lib import getData, loadDataFromCSV, timeStampToDate, saveCSV
from uAI_plot import plot_candleStick
from uAI_Finance import Finance
from uAI_typedef import *

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username='jbernardoCruz', api_key='wLC85rHSk0rclw6QbkAn')

In [61]:
'''GET DATA'''

timeFrame = 15
coin = 'ETHUSD'
fileName = 'data_test.csv'

data = getData(str(timeFrame) + 'm', coin, 1000)
saveCSV(data, fileName)

Requesting Data: https://api.bitfinex.com/v2/candles/trade:15m:tETHUSD/hist?&limit=1000

Data successfully acquired

Saving .csv file...

File saved: data_test.csv



In [62]:
'''LOAD DATA'''

csvData = loadDataFromCSV(fileName, [TIME, OPEN, HIGH, LOW, CLOSE, VOL])
finance = Finance(csvData)
data = finance.data


Finance data set.


In [63]:
'''PLOT DATA'''

#Price
price_graph = go.Candlestick(x=data[TIME], open=data[OPEN], high=data[HIGH], low=data[LOW],close=data[CLOSE], name=str(coin))

rsiUp = []
rsiLow = []
for i in range(len(finance.rsi)):
    rsiUp.append(70)
    rsiLow.append(30)

    
colors = ['rgb(67,67,67)', 'rgb(49,130,189)']
#RSI
rsi_graph = go.Scatter(x = data[TIME], y = finance.RSI(),  name='RSI',  mode='lines', line=dict(color=colors[0]))
rsiUp_graph = go.Scatter(x = data[TIME], y = rsiUp, name='', mode='lines', line=dict(color=colors[1]))
rsiLow_graph = go.Scatter(x = data[TIME], y = rsiLow, name='', mode='lines', line=dict(color=colors[1]))

#Stochastic RSI
s_rsi = finance.S_RSI()
#S_RSI 
s_rsi_k_graph = go.Scatter(x = data[TIME], y = s_rsi[S_RSI_K],  name='Stochastic RSI')
s_rsi_d_graph = go.Scatter(x = data[TIME], y = s_rsi[S_RSI_D],  name='Stochastic RSI')

fig = plotly.tools.make_subplots(rows=3, cols=1, shared_xaxes=True)
fig.append_trace(price_graph, 1, 1)
fig.append_trace(rsi_graph, 2, 1)
fig.append_trace(rsiUp_graph, 2, 1)
fig.append_trace(rsiLow_graph, 2, 1)
fig.append_trace(s_rsi_k_graph, 3, 1)
fig.append_trace(s_rsi_d_graph, 3, 1)

layout = go.Layout(xaxis = dict(rangeslider = dict(visible = False)),
                   yaxis1=dict(domain=[0.5, 1]), 
                   yaxis2=dict(domain=[0.2, 0.5]),      
                   yaxis3=dict(domain=[0, 0.2]),
                   autosize=False, width=1000,height=900, title='uAI')
    
fig['layout'].update(layout)
              
py.iplot(fig, filename='simple_candlestick')


This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~jbernardoCruz/0 or inside your plot.ly account where it is named 'simple_candlestick'
